In [5]:
def trade(usdt_pool: float, mind_pool: float, fee: float, trade_amount: float, is_buy: bool) -> float:
    k = usdt_pool * mind_pool
    if is_buy:
        new_usdt_pool = usdt_pool + trade_amount * (1 - fee)
        new_mind_pool = k / new_usdt_pool
        mind_bought = mind_pool - new_mind_pool
        return mind_bought
    else:
        new_mind_pool = mind_pool + trade_amount * (1 - fee)
        new_usdt_pool = k / new_mind_pool
        usdt_bought = usdt_pool - new_usdt_pool
        return usdt_bought


In [6]:
def calculate_new_price(x, y, r, b, s, t, K, M):
    """
    Calculate new price using the formula:

    p_new = x_final / y_final

    Parameters:
    -----------
    x : float
        USDT in pool
    y : float
        MIND tokens in pool
    r : float
        Fee rate (e.g., 0.003 for 0.3%)
    b : float
        Buyback dollars per day
    s : float
        Speculative buy pressure dollars per day
    t : float
        Speculative sell pressure dollars per day
    K : float
        Miner tokens sold per day
    M : float
        Miner tokens staked (locked) per day

    Returns:
    --------
    float
        New price (USDT per MIND token)
    """
    # Net dollar flow
    net_dollar_flow = b + s - t

    # Net token flow (K tokens sold minus M tokens staked)
    net_token_flow = K - M

    # Calculate y_final (denominator)
    denominator_part = (y * r * net_dollar_flow) / (x + r * net_dollar_flow)
    y_final = y - denominator_part + r * net_token_flow

    # Calculate x_final (numerator)
    x_adjustment = ((x + net_dollar_flow) * r * net_token_flow) / y_final
    x_final = x + net_dollar_flow - x_adjustment

    # Calculate new price
    p_new = x_final / y_final

    return p_new

In [7]:
# Example usage
if __name__ == "__main__":
    # Initial pool state
    x = 300_000  # 300k USDT
    y = 300_000  # 300k MIND tokens
    r = 0.003    # 0.3% fee
    
    # Daily flows
    b = 5_000    # $5k buybacks
    s = 10_000   # $10k speculative buying
    t = 8_000    # $8k speculative selling
    K = 5_000    # 5k tokens sold by miners
    M = 7_000    # 7k tokens staked by miners
    
    # Calculate new price
    new_price = calculate_new_price(x, y, r, b, s, t, K, M)
    
    print(f"Initial price: ${x/y:.4f}")
    print(f"New price: ${new_price:.4f}")
    print(f"Price change: {(new_price/(x/y) - 1)*100:.2f}%")

Initial price: $1.0000
New price: $1.0234
Price change: 2.34%


Data that we need an idea of.

For buybacks: How much we make in a day?
Speculative buying; This probably starts high and goes lower over time, but very hard to model
Speculative selling; Similar to the above but likley goes in the other direction
Tokens sold by miners: Inflation rate and a guess of the amount that is sold in percentage, also old miners selling and leaving
Tokens bought by miners: Miners buying to stake


Assumptions; 500 bought a day  
10k tokens staked to mine  
Miner every 10 days comes online  
Miner leaves every 20 days  
1k tokens given to miners a day  
Half of those are sold, half are kept  

In [8]:
b = 500
s = 0
t = 0
K = 1_000 * 0.5 + 10_000 * 0.05 # 1k tokens given out, half sold
M = 10_000 * 0.1 # 10k tokens staked to mine, new miner every 10 days

In [ ]:
new_price = calculate_new_price(x, y, r, b, s, t, K, M)

print(f"Initial price: ${x/y:.4f}")
print(f"New price: ${new_price:.4f}")
print(f"Price change: {(new_price/(x/y) - 1)*100:.2f}%")


Initial price: $1.0000
New price: $1.0017
Price change: 0.17%
